In [1]:
!pip install -U scikit-learn 

     |████████████████████████████████| 11.1MB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 73.3MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 78.8MB/s eta 0:00:01
     |████████████████████████████████| 307kB 84.9MB/s eta 0:00:01
ERROR: mlflow 2.10.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: fosforio 1.0.1 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement snowflake-snowpark-python<2,>=1.4.0, but you'll have snowflake-snowpark-python 1.0.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.1 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 3.0.3 which is incomp

In [ ]:
import time
from snowflake.snowpark.session import Session
import configparser
 
import warnings
warnings.filterwarnings("ignore")
 
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")
 
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}
 
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session
 
session = snowflake_connector(connection_parameters)

In [ ]:
df_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [ ]:
df_or

In [ ]:

df_date = df_or[["Date","Location_type"]]
df_date

df_date = df_date.iloc[:100, :]
df_date

In [ ]:
#df_or100 = df_or.iloc[:100, :]
#df_or100
#df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()

#df_or100= df_or100.reset_index()
#df_or100

In [ ]:
#df_or
#df_test_or= df_test_or.reset_index()
#df_test_or.head()
#df_or
#df_test_or

In [ ]:
import pandas as pd
import numpy as np
import sklearn 

#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns


from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
# train dataset
#df = pd.read_csv("1.FMCG_data_original.csv")

# test dataset
#df_test = pd.read_csv("1.FMCG_data_original.csv")

In [ ]:
df_or.nunique().sort_values(ascending=False)

In [ ]:

df_or


In [ ]:
df_or.drop(columns=['Ware_house_ID','WH_Manager_ID','Date'], inplace = True)
df_or['workers_num'].median()
df_or['approved_wh_govt_certificate'].mode()
df_or['approved_wh_govt_certificate'].fillna(df_or['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_or.isnull().sum()
df_or.nunique().sort_values(ascending= True)

In [ ]:
for i in df_or.columns[df_or.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_or[i].unique()))
    print(df_or[i].value_counts())
    print(' ')

In [ ]:
def count_outliers(df_or):
    outliers={}
    for i in df_or.columns:
        if pd.api.types.is_numeric_dtype(df_or[i]):
            q1 = df_or[i].quantile(0.25)
            q3 = df_or[i].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5*iqr
            upper = q3 + 1.5*iqr
            outliers[i] = df_or[(df_or[i] < lower) | (df_or[i] > upper)].shape[0]
        else:
            pass
    return outliers

In [ ]:
outlier_counts = count_outliers(df_or)
outlier_counts

In [ ]:
df_or['product_wg_ton'].corr(df_or['flood_proof'])
df_or['product_wg_ton'].corr(df_or['flood_impacted'])
df_or.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)

In [ ]:
def remove_outliers(df_or):
    num_df = df_or.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_or.loc[df_no_outliers.index]

In [ ]:
df_or = remove_outliers(df_or)
df_or.shape

In [ ]:
df_or

In [ ]:
df_test_or.nunique().sort_values(ascending=False)

In [ ]:
df_test_or.drop(columns=['Ware_house_ID', 'WH_Manager_ID','Date'], inplace = True)
print(len(df_test_or[df_test_or.duplicated]))

In [ ]:
df_test_or

In [ ]:
# print percentage of null values in each column

null_values_features = [i for i in df_test_or.columns if df_test_or[i].isnull().sum()>1]
for i in null_values_features:
    print(i, np.round(df_test_or[i].isnull().mean()*100, 2), "% missing values")

In [ ]:
df_test_or.drop(columns=['wh_est_year'], inplace = True)
df_test_or['workers_num'].median()
df_test_or['workers_num'].fillna(df_test_or['workers_num'].median(), inplace=True)
df_test_or['approved_wh_govt_certificate'].mode()
df_test_or['approved_wh_govt_certificate'].fillna(df_test_or['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_test_or.isnull().sum()
df_test_or.nunique().sort_values(ascending= True)

In [ ]:
for i in df_test_or.columns[df_test_or.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_test_or[i].unique()))
    print(df_test_or[i].value_counts())
    print(' ')

In [ ]:
df_test_or['product_wg_ton'].corr(df_test_or['storage_issue_reported'])

In [ ]:
df_test_or.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)

In [ ]:
def remove_outliers(df_test_or):
    num_df = df_test_or.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_test_or.loc[df_no_outliers.index]

In [ ]:
df_test_or = remove_outliers(df_test_or)
df_test_or

In [ ]:
#df_or_dt = df_or[['Date']].astype(str)
#df_or_dt.head()


#df_or_dt


In [ ]:
#df_test_or_dt = df_test_or[['Date']].astype(str)
#df_test_or_dt.head()

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_or1 = df_or.drop(['Date'], axis=1)
#df_or1.head()

In [ ]:
#df_test_or1 = df_test_or.drop(['Date'], axis=1)
#df_test_or1.head()

In [ ]:
# Numerical columns & categorical columns in train dataset

num_columns_train=[i for i in df_or.columns if df_or[i].dtypes != 'O']
cat_columns_train=[i for i in df_or.columns if df_or[i].dtypes == 'O']

# Numerical columns & categorical columns in test dataset

num_columns_test=[i for i in df_test_or.columns if df_test_or[i].dtypes != 'O']
cat_columns_test=[i for i in df_test_or.columns if df_test_or[i].dtypes == 'O']

In [ ]:
for i in cat_columns_train:
    df_or[i] = LabelEncoder().fit_transform(df_or[i])

for i in cat_columns_test:
    df_test_or[i] = LabelEncoder().fit_transform(df_test_or[i])

In [ ]:
# display number of unique values in each column
df_or.nunique().sort_values(ascending=False)

In [ ]:
columns_with_more_than_two_unique_values_train = df_or.columns[ df_or.nunique()>2]

In [ ]:
columns_with_more_than_two_unique_values_test = df_test_or.columns[df_test_or.nunique()>2]

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_or2 = pd.concat([df_or_dt,df_or1], ignore_index=True, sort=False)
#df_or2.head()

In [ ]:
df_or

In [ ]:
#df['Location_type'] = df['Location_type'].replace([0,1])
#df

In [ ]:
X_train = df_or.drop(['product_wg_ton','wh_est_year', 'WH_regional_zone'], axis=1)
Y_train = df_or['product_wg_ton']

In [ ]:
X_train.columns

In [ ]:
# df_or_dt - only date column

# df_or1 - all columns except date column 

# df_test_or_dt - only date column

# df_test_or1 - all columns except date column

In [ ]:
#df_test_or2 = pd.concat([df_test_or_dt,df_test_or1], ignore_index=True, sort=False)

In [ ]:
#df_test_or2.head()

In [ ]:
df_test_or

In [ ]:
X_test = df_test_or.drop(['product_wg_ton','WH_regional_zone'], axis=1)
X_test.head()

In [ ]:
X_test.columns

In [ ]:
X_test = X_test.iloc[:500, :]
X_test.columns

In [ ]:
#Linear Regression
LRmodel = LinearRegression()

In [ ]:
LRmodel.fit(X_train, Y_train)

In [ ]:
LRmodel.fit(X_train, Y_train)

LR_y_predicted = LRmodel.predict(X_test)
LR_y_predicted


In [ ]:
predictions_location_nochange = pd.DataFrame(LR_y_predicted, columns =['product_wg_ton_pred_location_nochange']) 
predictions_location_nochange

In [ ]:
df_location = df_or[["Location_type"]]
df_location

df_location = df_location.iloc[:100, :]
df_location

In [ ]:
ch = predictions_location_nochange.iloc[:500, :]
ch

In [ ]:
allcols = df_test_or.iloc[:500, :]
allcols.isnull().sum()

In [ ]:
all = pd.concat([df_date, predictions_location_nochange,allcols], axis=1, join='outer')
all.isnull().sum()

all = all.dropna()
all.isnull().sum()

In [ ]:
all

In [ ]:
df_nochange_location=session.createDataFrame(
        all.values.tolist(),
        schema=all.columns.tolist())


In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_fun")

In [ ]:

allcols = df_test_or.iloc[:100, :]
allcols.isnull().sum()

In [ ]:
final_reco = pd.concat([two_cols, allcols], axis=1, join='outer')
final_reco

final_reco = final_reco.iloc[:99, :]
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())



In [ ]:
final_reco = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
df_test_or = df_test_or.iloc[:100, :]
df_test_or.columns

In [ ]:
df_test_or.isnull().sum()

In [ ]:
df_test_or

In [ ]:
predictions_location_nochange = predictions_location_nochange.reset_index()
predictions_location_nochange

df_test_or

In [ ]:
df_test_or = df_test_or.reset_index()
df_test_or

In [ ]:
final_reco = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_rev")

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION_rev")

In [ ]:
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or
df_test_or100 = df_test_or.iloc[:100, :]
df_test_or100

In [ ]:
gg= pd.concat([df_test_or100, predictions_location_nochange], axis=1, join='outer')
gg.head(100)

In [ ]:
df_nochange_location=session.createDataFrame(
        gg.values.tolist(),
        schema=gg.columns.tolist())

In [ ]:
df_nochange_location

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("NOCHANGE_LOCATION")

In [ ]:
predictions_location_nochange = predictions_location_nochange.reset_index()
predictions_location_nochange

In [ ]:
df_test_or = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_or

In [ ]:
df_merged = pd.concat([df_test_or, predictions_location_nochange], ignore_index=True, sort=False)

In [ ]:
df_merged

In [ ]:
df_test_or= df_test_or.reset_index()
df_test_or.head()

In [ ]:
df_test_or = df_test_or.iloc[:100, :]
df_test_or

In [ ]:
final_reco1 = pd.merge(df_test_or,predictions_location_nochange,on='index',how='inner')

In [ ]:
final_reco1

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco1.values.tolist(),
        schema=final_reco1.columns.tolist())


In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("PRED_NOCHANGE_LOCATION")

In [ ]:
#####WITH CHANGED LOCATION 

In [ ]:
!pip install -U scikit-learn

In [ ]:
import time
from snowflake.snowpark.session import Session
import configparser
 
import warnings
warnings.filterwarnings("ignore")
 
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")
 
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}
 
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session
 
session = snowflake_connector(connection_parameters)

In [ ]:
df_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [ ]:
df_CL.head()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_CL)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df_CL['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df_CL.to_csv('CPG_DATA_LOCATION_CHANGE1.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")


In [ ]:
df_CL.head()

In [ ]:
df_test_CL = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_test_CL)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df_test_CL['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df_test_CL.to_csv('CPG_DATA_LOCATION_CHANGE1.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")

In [ ]:
df_test_CL

In [ ]:
df_CL.head()


In [ ]:
df_test_CL.head()

In [ ]:
df_CL= df_CL.reset_index()
df_CL.head()

In [ ]:
df_test_CL= df_test_CL.reset_index()
df_test_CL.head()

In [ ]:
import pandas as pd
import numpy as np
import sklearn 

#import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns


from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
df_CL.nunique().sort_values(ascending=False)
df_CL.drop(columns=['Ware_house_ID','WH_Manager_ID','Date'], inplace = True)
df_CL['workers_num'].median()
df_CL['approved_wh_govt_certificate'].mode()
df_CL['approved_wh_govt_certificate'].fillna(df_CL['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_CL.isnull().sum()
df_CL.nunique().sort_values(ascending= True)

In [ ]:
for i in df_CL.columns[df_CL.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_CL[i].unique()))
    print(df_CL[i].value_counts())
    print(' ')

In [ ]:
def count_outliers(df_CL):
    outliers={}
    for i in df_CL.columns:
        if pd.api.types.is_numeric_dtype(df_CL[i]):
            q1 = df_CL[i].quantile(0.25)
            q3 = df_CL[i].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5*iqr
            upper = q3 + 1.5*iqr
            outliers[i] = df_CL[(df_CL[i] < lower) | (df_CL[i] > upper)].shape[0]
        else:
            pass
    return outliers

In [ ]:
outlier_counts = count_outliers(df_CL)
outlier_counts

In [ ]:
df_CL['product_wg_ton'].corr(df_CL['flood_proof'])
df_CL['product_wg_ton'].corr(df_CL['flood_impacted'])
df_CL.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)


In [ ]:
def remove_outliers(df_CL):
    num_df = df_CL.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_CL.loc[df_no_outliers.index]

In [ ]:
df_CL = remove_outliers(df_CL)
df_CL.shape

In [ ]:
df_test_CL.nunique().sort_values(ascending=False)

In [ ]:
df_test_CL.drop(columns=['Ware_house_ID', 'WH_Manager_ID'], inplace = True)
print(len(df_test_CL[df_test_CL.duplicated]))



In [ ]:
null_values_features = [i for i in df_test_CL.columns if df_test_CL[i].isnull().sum()>1]
for i in null_values_features:
    print(i, np.round(df_test_CL[i].isnull().mean()*100, 2), "% missing values")


In [ ]:
df_test_CL.drop(columns=['wh_est_year'], inplace = True)
df_test_CL['workers_num'].median()
df_test_CL['workers_num'].fillna(df_test_CL['workers_num'].median(), inplace=True)
df_test_CL['approved_wh_govt_certificate'].mode()
df_test_CL['approved_wh_govt_certificate'].fillna(df_test_CL['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_test_CL.isnull().sum()
df_test_CL.nunique().sort_values(ascending= True)

In [ ]:
for i in df_test_CL.columns[df_test_CL.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_test_CL[i].unique()))
    print(df_test_CL[i].value_counts())
    print(' ')

In [ ]:
df_test_CL['product_wg_ton'].corr(df_test_CL['storage_issue_reported'])
df_test_CL.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)



In [ ]:
def remove_outliers(df_test_CL):
    num_df = df_test_CL.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_test_CL.loc[df_no_outliers.index]

In [ ]:
df_test_CL = remove_outliers(df_test_CL)
df_test_CL.head()

In [ ]:
num_columns_train=[i for i in df_CL.columns if df_CL[i].dtypes != 'O']
cat_columns_train=[i for i in df_CL.columns if df_CL[i].dtypes == 'O']

# Numerical columns & categorical columns in test dataset

num_columns_test=[i for i in df_test_CL.columns if df_test_CL[i].dtypes != 'O']
cat_columns_test=[i for i in df_test_CL.columns if df_test_CL[i].dtypes == 'O']




In [ ]:
for i in cat_columns_train:
    df_CL[i] = LabelEncoder().fit_transform(df_CL[i])

for i in cat_columns_test:
    df_test_CL[i] = LabelEncoder().fit_transform(df_test_CL[i])
    



In [ ]:
df_CL.nunique().sort_values(ascending=False)
columns_with_more_than_two_unique_values_train = df_CL.columns[df_CL.nunique()>2]
columns_with_more_than_two_unique_values_test = df_test_CL.columns[df_test_CL.nunique()>2]


In [ ]:
X_train = df_CL.drop(['product_wg_ton','wh_est_year', 'WH_regional_zone'], axis=1)
Y_train = df_CL['product_wg_ton']
X_train.columns


In [ ]:
X_test =  df_test_CL.drop(['product_wg_ton','WH_regional_zone','Date'], axis=1)
X_test.head()
X_test.columns

In [ ]:
X_train.columns

In [ ]:
X_train= X_train.reset_index()

In [ ]:
X_train.columns

In [ ]:
X_test1 = X_test.iloc[:100, :]
X_test1.columns

In [ ]:
X_train.columns

In [ ]:
X_train = X_train.drop(['level_0'], axis=1)
X_train.columns

In [ ]:
GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,Y_train)
GB_predicted = GBmodel.predict(X_test1)
GB_predicted

In [ ]:
predictions_location_change = pd.DataFrame(GB_predicted, columns =['product_wg_ton_pred_location_change']) 
predictions_location_change


In [ ]:
predictions_location_change = predictions_location_change.reset_index()
predictions_location_change


In [ ]:
predictions_location_change.shape

In [ ]:
#df_test_CL= df_test_CL.reset_index()
#df_test_CL.head()

df_test_CL = df_test_CL.iloc[:100, :]
df_test_CL

In [ ]:
final_reco1 = pd.merge(df_test_or,predictions_location_change,on='index',how='inner')



In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco1.values.tolist(),
        schema=final_reco1.columns.tolist())



In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_NOCHANGE_LOCATION0")

In [ ]:
final_reco = pd.merge(df_test_CL,predictions_location_change,on='index',how='inner')

In [ ]:
df_change_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())




In [ ]:
df_change_location.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_CHANGE_LOCATION12")

In [ ]:
predictions_nolocationchange = pd.DataFrame(LR_y_predicted, columns =['product_wg_ton_pred_nolocationchange']) 
predictions_nolocationchange

In [ ]:
predictions_nolocationchange = predictions_nolocationchange.reset_index()

predictions_nolocationchange

In [ ]:
final_reco = pd.merge(df_test_or,predictions_nolocationchange,on='index',how='inner')
final_reco.head()

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

In [ ]:
df_test_or.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE11")


In [ ]:
final_reco.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
final_reco

In [ ]:
df_test_or.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE11")

In [ ]:

predictions_location_nochange = predictions_nolocationchange.reset_index()

predictions_location_nochange

In [ ]:
df_test_location= df_test.reset_index()
df_test_location

In [ ]:
final_reco = pd.merge(df_test,predictions_location,on='index',how='inner')

In [ ]:
df_test.write.mode("overwrite").save_as_table("CPG_DATA_PRED_NOCHANGE1")
predictions_location.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
predictions_location.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
Y_test = df_test['product_wg_ton']
Y_test.head()

In [ ]:
# r2 score
lr_r2 = r2_score(Y_test, LR_y_predicted).round(3)
print('R2 score:  ', lr_r2)

# root mean squared error (RMSE)
lr_rmse = np.sqrt(mean_squared_error(Y_test, LR_y_predicted)).round(3)
print('Root Mean Squared Error:  ', lr_rmse) 

# mean absolute error (MAE)
lr_mae = mean_absolute_error(Y_test, LR_y_predicted).round(3)
print('Mean Absolute Error:  ', lr_mae)

In [ ]:
DTmodel = DecisionTreeRegressor(random_state = 0)

In [ ]:

DTmodel.fit(X_train, Y_train) 

In [ ]:
DT_predicted = DTmodel.predict(X_test)

In [ ]:
DT_predicted

In [ ]:
# r2 score
dt_r2 = r2_score(Y_test, DT_predicted).round(3)
print('R2 score:  ', dt_r2)

# root mean squared error (RMSE)
dt_rmse = np.sqrt(mean_squared_error(Y_test, DT_predicted)).round(3)
print('Root Mean Squared Error:  ', dt_rmse) 

# mean absolute error (MAE)
dt_mae = mean_absolute_error(Y_test, DT_predicted).round(3)
print('Mean Absolute Error:  ', dt_mae)

In [ ]:
RFmodel = RandomForestRegressor(n_estimators = 100, max_depth = 4, random_state = 0) 
RFmodel.fit(X_train, Y_train)
RF_predicted = RFmodel.predict(X_test)
RF_predicted

In [ ]:
# r2 score
rf_r2 = r2_score(Y_test, RF_predicted).round(3)
print('R2 score:  ', rf_r2)

# root mean squared error (RMSE)
rf_rmse = np.sqrt(mean_squared_error(Y_test, RF_predicted)).round(3)
print('Root Mean Squared Error:  ', rf_rmse) 

# mean absolute error (MAE)
rf_mae = mean_absolute_error(Y_test, RF_predicted).round(3)
print('Mean Absolute Error:  ', rf_mae)

In [ ]:

GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,Y_train)
GB_predicted = GBmodel.predict(X_test)
GB_predicted

In [ ]:
gb_r2 = r2_score(Y_test, GB_predicted).round(3)
print('R2 score:  ', gb_r2)

# root mean squared error (RMSE)
gb_rmse = np.sqrt(mean_squared_error(Y_test, GB_predicted)).round(3)
print('Root Mean Squared Error:  ', gb_rmse) 

# mean absolute error (MAE)
gb_mae = mean_absolute_error(Y_test, GB_predicted).round(3)
print('Mean Absolute Error:  ', gb_mae)

In [ ]:
ax1 = plt.subplot2grid((2, 3), (0, 0))
plt.scatter(Y_test, LR_y_predicted, color='#ffdab9', alpha=0.3, label='Actual')
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f', label='Predicted')
plt.title('Linear Regression')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()


ax1 = plt.subplot2grid((2, 3), (0, 1))
plt.scatter(Y_test, DT_predicted, color='#ffdab9', alpha=0.3)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f')
plt.title('Decision Tree')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
ax1 = plt.subplot2grid((2, 3), (1, 1))
plt.scatter(Y_test, GB_predicted, color='#ffdab9', alpha=0.3)  # alpha controls point transparency for better visualization
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--',color='#2f4f4f')
plt.title('Gradient Boosting')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

plt.show()

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df_or)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location_changed = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location_changed)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df['location_changed'] = location_changed

# Save the updated DataFrame back to a CSV file
df.to_csv('CPG_DATA_ORIGINAL.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")
